# Air Pollution Forecasting - Part 1
### EDA and Feature Engineering

## Why Forecast? 
Air pollution has long been held as one of the foremost threats to human health, especially in metropolitan areas throughout the world. Exposure to air pollution can lead to may lead to diseases such as asthma, lung cancer, or cardiovascular diseases and can prove to be fatal, not only by diseases but also by issues like low visibilty. Forecasting air pollution can thus help us take appropriate measures to tackle this issue.

## Why PM2.5?
The pollutant that affects people the most is particulate matter, usually abbreviated as PM and used as a measure of air pollution. Although particles with a diameter of 10 microns or less (≤PM10) can penetrate and embed deep in the lungs, the ones that are more harmful to health are those with a diameter of 2.5 microns or less (≤PM2.5). Sources of PM2.5 can be coal-fired power generation, smoke, or dusts. 

## About the Project
The Air Quality dataset for Bejing has been provided. This is a dataset that reports on the weather and the level of pollution each hour for five years at the US embassy in Bejing, China. This project focuses on multivariate analysis of data to forecast pollution levels for the next 24 hours.

The data for this project can be downloaded from **[kaggle](https://www.kaggle.com/datasets/rupakroy/lstm-datasets-multivariate-univariate)** and stored as a csv file in a 'data' folder.

According to **[IQAir](https://www.iqair.com/china/beijing)**, PM2.5 concentration in Bejing is currently **6.6 times the WHO annual air quality guideline value.** There are various AQI scales, but since the data is obtained from China, this project will refer to the local scale as shown in the image below. 

![PM2.5 levels Table](https://www.researchgate.net/profile/Chengcai-Li/publication/304400164/figure/tbl1/AS:668877105156097@1536484137117/AQI-and-air-pollution-levels-with-corresponding-PM-25-concentrations-Ministry-of.png "PM2.5 Levels")

## Importing the necessary Libraries

In [ ]:
# Importing the required libraries

# Libraries for reading the data and preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
from pandas.plotting import lag_plot

# Libraires for time series preprocessing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings("ignore")
import calendar

#Pep8
import jupyter_black
jupyter_black.load()

## Reading The Data

The data includes the date-time, the pollution called PM2.5 concentration, and the weather information.
It is curated and stored originally in **[UCI ML Repository.](https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data)**
The complete feature list in the raw data is as follows:

| Feature Name | Description |
| --- | --- |
| date | Date when the observation was captured |
| pollution | PM2.5 concentration (ug/m^3) |
| dew | Dew Point |
| temp | Temperature |
| press | Pressure (hPa) |
| wnd_dir |  Combined wind direction  |
| wnd_spd | Cumulated wind speed in m/s |
| snow | Cumulated hours of snow |
| rain | Cumulated hours of rain |

In [ ]:
air_quality_data = pd.read_csv("data/LSTM-Multivariate_pollution.csv")
air_quality_data.head(3)

In [ ]:
# Checking if date is the primary key of this table
len(air_quality_data["date"].unique())

In [ ]:
air_quality_data.dtypes

In [ ]:
air_quality_data["wnd_dir"].unique()

### Observations and Insights:   
- There are 43,800 rows and 9 columns
- Most of the columns are of numeric datatype. The columns `date` and `wnd_dir` are of the type 'object'. 
- The `date` column can be should be converted to appropriate datatype.
- The 4 unique values of `wnd_dir` can be cleaned and one-hot encoded for future use, if needed.
- The feature `pollution` is the target variable
- We can see that each of these observations is uniquely indexed by a corresponding 'date' value

## Exploratory Data Analysis

Some questions explored through EDA are:
- Frequency of the data
- PM2.5 concentration through different years 
- PM2.5 concentration othrough different months
- PM2.5 concentration through differnent hours
- Is there any pattern/trend?
- Does the data have seasonality?

### 1. Changing datatype of Date Column and setting it as index

In [ ]:
air_quality_data["date"] = pd.to_datetime(
    air_quality_data.date, infer_datetime_format="True"
)
air_quality_data.dtypes

In [ ]:
air_quality_data.set_index("date", inplace=True)
air_quality_data.head(2)

### 2. Cheking Missing data

There are no missing values.

In [ ]:
air_quality_data.isna().sum()

### 3. Summary Statistics
- The data provided seems correct when checked with the actual recorded [data](https://weatherspark.com/y/131055/Average-Weather-in-Beijing-China-Year-Round) of Bejing. 
- The values of snow and rain being more than 24hrs is justified as they are cumulative values
- The maximum pollution level is 994. On checking further, we see that this record is found in Jan 2012. It seems like an error but it is not and hence we won't be dropping it. [Refered here](https://qz.com/61694/chinas-nightmare-scenario-by-2025-air-quality-could-be-much-much-worse/) for checking.
- On an average, the pollution level is Bejing is 94.

In [ ]:
air_quality_data.describe()

In [ ]:
air_quality_data[air_quality_data["pollution"] >= 900]

### 4. Data Distribution
The features do not follow a normal distribution and normalization might be needed going further depending on the choice of algorithms.

In [ ]:
plt.figure(figsize=(10, 5))
numeric_df = air_quality_data[
    ["pollution", "dew", "temp", "wnd_spd", "snow", "rain", "press"]
]
for i, col in enumerate(list(numeric_df.columns.values)):
    plt.subplot(4, 2, i + 1)
    p = sns.distplot(numeric_df[col], color="r", kde=True, label="Values")
    plt.grid()
    plt.legend(loc="upper right")
    p.set(yticklabels=[])
    p.set(ylabel=None)
    plt.tight_layout()

### 5. Visualizing the target variable

In [ ]:
fig = plt.figure(figsize=(20, 7))

df = air_quality_data.reset_index()
fig = px.line(df, x="date", y="pollution")

fig.update_xaxes(
    rangeslider_visible=True,
    title_text="Date",
    rangeselector=dict(
        buttons=list(
            [
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=2, label="2y", step="year", stepmode="backward"),
                dict(count=3, label="3y", step="year", stepmode="backward"),
                dict(count=4, label="4y", step="year", stepmode="backward"),
                dict(step="all"),
            ]
        )
    ),
)
fig.update_yaxes(title_text="Pollution concentration (PM 2.5)")
fig.update_layout(
    title_text="Interactive chart of Air Pollution levels across years", title_x=0.5
)
fig.show()

#### 5.1 Creating month, day, year and hour columns for checking pollution levels at different granularities

In [ ]:
air_quality_data["day"] = air_quality_data.index.map(lambda x: x.day)
air_quality_data["month"] = air_quality_data.index.map(lambda x: x.month)
air_quality_data["year"] = air_quality_data.index.map(lambda x: x.year)
air_quality_data["hour"] = air_quality_data.index.map(lambda x: x.hour)


air_quality_data.head(2)

#### 5.2 Deep-diving into the pollution chart per year, month and day - Interactive Vizualization

In [ ]:
df_2010 = air_quality_data[air_quality_data.year == 2010]
df_2011 = air_quality_data[air_quality_data.year == 2011]
df_2012 = air_quality_data[air_quality_data.year == 2012]
df_2013 = air_quality_data[air_quality_data.year == 2013]
df_2014 = air_quality_data[air_quality_data.year == 2014]

fig = make_subplots(rows=5, cols=1)
fig.add_trace(
    go.Scatter(
        x=df_2010.reset_index()["date"],
        y=df.reset_index()["pollution"],
        name="pollution in 2010",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_2011.reset_index()["date"],
        y=df.reset_index()["pollution"],
        name="pollution in 2011",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_2012.reset_index()["date"],
        y=df.reset_index()["pollution"],
        name="pollution in 2012",
    ),
    row=3,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_2013.reset_index()["date"],
        y=df.reset_index()["pollution"],
        name="pollution in 2013",
    ),
    row=4,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_2014.reset_index()["date"],
        y=df.reset_index()["pollution"],
        name="pollution in 2014",
    ),
    row=5,
    col=1,
)
fig.update_layout(
    height=600, width=800, title_text="Pollution Level Each Year", title_x=0.5
)
fig.show()

- We can see that pollution levels across years follow the same pattern with spikes in the month of february, march, october, november and december
- The date on which the major spike occurs is 14th Feb for the year 2010, and 13th feb for other years

#### 5.3 Average pollution values at different granularities
- The pollution levels were lowest in the year 2012 and highest in 2013, but do not vary by much.
- End of october to early march denotes winter in Bejing. There is a increase in PM2.5 concentration during winter months compared to the summers because of heating used in many households.The lowest levels are in the months of April, May, August and September
- The pollution concentration is highest on saturdays, but not by much. Day of week doesn't seem to affect pollution levels a lot
- Interestingly, the pollution levels are high during the late evenings and nights as compared to the standard working hours. It can be attributed to trucks travelling during the nights or the atmospheric conditions.

In [ ]:
plt.rc("figure", figsize=(12, 8))
fig, axes = plt.subplots(nrows=2, ncols=2)
air_quality_data.groupby(by="year").mean()["pollution"].plot(kind="bar", ax=axes[0, 0])
axes[0, 0].set_ylabel("Mean PM 2.5 Concentration (μg/m3)")
axes[0, 0].set_title("Air Pollution by Year")

air_quality_data.groupby(by="month").mean()["pollution"].plot(kind="bar", ax=axes[0, 1])
axes[0, 1].set_xticklabels(calendar.month_name[1:13])
axes[0, 1].set_ylabel("Mean PM 2.5 Concentration (μg/m3)")
axes[0, 1].set_title("Air Pollution by Month")

air_quality_data.groupby(by="hour").mean()["pollution"].plot(kind="bar", ax=axes[1, 1])
axes[1, 1].set_ylabel("Mean PM 2.5 Concentration (μg/m3)")
axes[1, 1].set_title("Air Pollution by hours")

air_quality_data_copy = air_quality_data
air_quality_data_copy = air_quality_data_copy.reset_index()
air_quality_data_copy["day_of_week"] = air_quality_data_copy.date.dt.dayofweek
air_quality_data_copy = air_quality_data_copy.set_index("date")
air_quality_data_copy.groupby(by="day_of_week").mean()["pollution"].plot(
    kind="bar", ax=axes[1, 0]
)
axes[1, 0].set_ylabel("Mean PM 2.5 Concentration (μg/m3)")
axes[1, 0].set_title("Air Pollution by days")
axes[1, 0].set_xticklabels(calendar.day_name[0:7])

plt.tight_layout()
plt.show()

### 6. Analyzing percentage of days per each Air Classification Bracket, broken down by years

- We can see that the least number of days with good air quality was in the year 2013.
- The distribution of days is fairly similar through the years.

In [ ]:
plt.rc("figure", figsize=(10, 10))

fig, ax = plt.subplot_mosaic("ABCDE")

years = [2010, 2011, 2012, 2013, 2014]
plots = ["A", "B", "C", "D", "E"]

x = 0

for i in years:
    Pie_y_temp = air_quality_data[air_quality_data.year == i]
    Pie_y = Pie_y_temp.pollution.resample("D").mean()

    good = len([ii for ii in Pie_y.dropna() if ii <= 35])
    moderately = len([ii for ii in Pie_y.dropna() if ii > 35 and ii <= 75])
    lightly = len([ii for ii in Pie_y.dropna() if ii > 75 and ii <= 115])
    medically = len([ii for ii in Pie_y.dropna() if ii > 115 and ii <= 150])
    heavily = len([ii for ii in Pie_y.dropna() if ii > 150 and ii <= 250])
    severely = len([ii for ii in Pie_y.dropna() if ii > 350])

    sizes = [good, lightly, moderately, medically, heavily, severely]
    labels = [
        "Good",
        "Moderately polluted",
        "Lightly Polluted",
        "Medically Polluted",
        "Heavily Polluted",
        "Severely Polluted",
    ]

    colours = {
        "Good": "green",
        "Moderately polluted": "yellow",
        "Lightly Polluted": "orange",
        "Medically Polluted": "red",
        "Heavily Polluted": "purple",
        "Severely Polluted": "brown",
    }

    plot = plots[x]
    x = x + 1

    ax[plot].pie(sizes, autopct="%1.1f%%", colors=[colours[key] for key in labels])
    ax[plot].set_title("Year " + str(i))
fig.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0)
fig.legend(
    labels,
    frameon=False,
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    labelspacing=0,
    handletextpad=0.1,
    fontsize=12,
)
fig.show()

### 7. Visualizing Other Variables along with the target variable to identify any patterns

In [ ]:
fig = make_subplots(rows=6, cols=1)
df = air_quality_data
fig.add_trace(
    go.Scatter(
        x=df.reset_index()["date"], y=df.reset_index()["pollution"], name="pollution"
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df.reset_index()["date"], y=df.reset_index()["press"], name="Pressure"
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df.reset_index()["date"], y=df.reset_index()["wnd_spd"], name="wind_speed"
    ),
    row=3,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df.reset_index()["date"], y=df.reset_index()["temp"], name="temperature"
    ),
    row=4,
    col=1,
)

fig.add_trace(
    go.Scatter(x=df.reset_index()["date"], y=df.reset_index()["rain"], name="Rain"),
    row=5,
    col=1,
)

fig.add_trace(
    go.Scatter(x=df.reset_index()["date"], y=df.reset_index()["snow"], name="Snow"),
    row=6,
    col=1,
)
fig.update_layout(
    height=600,
    width=800,
    title_text="Relationship between pollution and weather across years",
    title_x=0.5,
)
fig.show()

### 8. Correlation Plot to check for any strong relationships

- None of the variables are highly correlated to the target variables when considered individually
- We can however see a strong correlation between dew, pressure and temperature

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(numeric_df.corr(), annot=True, cmap="RdBu_r")

### 9. Decomposing the Target Time Series 

This allows us to check for trend, seasonality and residuals. 

In [ ]:
# Extracting Monthly trend
plt.rc("figure", figsize=(10, 10))
result = seasonal_decompose(air_quality_data.pollution, model="additive", period=730)
result.plot();

### 10. Plotting Moving Averages to check trend at various levels

This is a raw approach to get the trend without using the `seasonal_decompose()` funtion. 

In [ ]:
mvg_avg = air_quality_data.copy()
mvg_avg["yearly_pollution"] = air_quality_data["pollution"].rolling(8760).mean()
mvg_avg["monthly_pollution"] = air_quality_data["pollution"].rolling(730).mean()
mvg_avg["weekly_pollution"] = air_quality_data["pollution"].rolling(168).mean()

In [ ]:
plt.rc("figure", figsize=(10, 10))
fig, ax = plt.subplot_mosaic("AB;CC")

ax["A"].plot(air_quality_data["pollution"], label="Pollution TS")
ax["A"].plot(mvg_avg["yearly_pollution"], label="Yearly Moving Average")
ax["A"].legend(loc="upper right")

ax["B"].plot(air_quality_data["pollution"], label="Pollution TS")
ax["B"].plot(mvg_avg["monthly_pollution"], label="Monthly Moving Average")
ax["B"].legend(loc="upper right")

ax["C"].plot(air_quality_data["pollution"], label="Pollution TS")
ax["C"].plot(mvg_avg["weekly_pollution"], label="Weekly Moving Average")
ax["C"].legend(loc="upper right")

plt.show()

#### Observations and Insights:   
- The data is quite clean with no missing values
- The features are not normally distributed
- When considered independently, no variables strongly correlate with the target variable and that can be seen from the correlation matrix and the graph showing relationship between weather and pollution
- There is a very similar pattern followed through the years by the target variable
- There is some seasonality in the data as per graphs
- Some features can be created to make more sense out of the data

## Feature Engineering

#### One-Hot Encoding the wind direction variable

In [ ]:
air_quality_data.columns

In [ ]:
air_quality_data = pd.concat(
    [air_quality_data, pd.get_dummies(air_quality_data["wnd_dir"], prefix="wnd_dir")],
    axis=1,
)
air_quality_data.drop(["wnd_dir"], axis=1, inplace=True)
air_quality_data.head()

#### Checking the Correlation of the target with itself to identify relevant lags
- The PACF plot shows that no lags apart from the first lag have direct effect on the currect value. 
- We will thus be considering the first lag while creating new feature.

In [ ]:
plt.rc("figure", figsize=(5, 3))
plot_pacf(air_quality_data["pollution"], lags=30, method="ols")
plt.tight_layout()
plt.show()

#### Create Lag for pollution

In [ ]:
# Shift the pollution to create the output variable
air_quality_data["Lag1_pollution"] = air_quality_data["pollution"].shift(1)

# Drop the last row (it has no value for predicted pollution)
air_quality_data = air_quality_data.drop(air_quality_data.head(1).index)

air_quality_data.head(3)

#### Calculating daily averages for target variable and creating its lag

We can see that only 1 lag of `Daily_Avg_Pollution` is significant and hence I am creating a new feature with one shift.

In [ ]:
air_quality_data["Daily_Avg_Pollution"] = air_quality_data.groupby(
    ["day", "month", "year"]
)["pollution"].transform("mean")

In [ ]:
plt.rc("figure", figsize=(5, 3))
plot_pacf(air_quality_data["Daily_Avg_Pollution"], lags=20, method="ols");

In [ ]:
air_quality_data["Lag1_daily_avg_pollution"] = air_quality_data[
    "Daily_Avg_Pollution"
].shift(1)

# Drop the first row (it has no value for predicted pollution)
air_quality_data = air_quality_data.drop(air_quality_data.head(1).index)
air_quality_data.columns

#### Create additional temporal features
I have already created hour, month, year and day features. The code below shows creation of `quarter`

In [ ]:
air_quality_data = air_quality_data.reset_index()
air_quality_data["quarter"] = air_quality_data["date"].dt.quarter
air_quality_data.columns

#### Saving the new file

In [ ]:
air_quality_data.to_csv("data/air_quality_data.csv", index=False)